In [1]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [45]:
df = pd.read_csv("D:/python/py-adult-data-analysis/data/adult.csv")
df.replace("?", np.nan, inplace=True)
df.dropna(inplace=True)
df.rename(columns={"Unnamed: 0": "index"}, inplace=True)
df.set_index("index", inplace=True)
valid_salary = ["<=50K", ">50K"]
if not df["salary"].isin(valid_salary).all():
    raise ValueError("salary is not valid")
if df["salary K$"].isna().any():
    raise ValueError("salary K$ not correct")
df

,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
index,,,,,,,,,,,,
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25
...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,38,United-States,<=50K,36
32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,>50K,173
32558,58,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,40,United-States,<=50K,40


# Task 1
Print the count of men and women in the dataset.

In [84]:
gender_count = df["sex"].value_counts()
gender_count

sex
Male      20380
Female     9782
Name: count, dtype: int64

# Task 2
Find the average age of men in dataset

In [83]:
average_age_man = df[df["sex"] == "Male"]["age"].mean()
average_age_man

np.float64(39.18400392541707)

# Task 3
Get the percentage of people from Poland (native-country)

In [82]:
polish_person = (df[df["native-country"] == "Poland"].shape[0] / df.shape[0]) * 100
polish_person

0.18566408063125786

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [80]:
high_earners_stats = df[df["salary K$"] > 50]["age"].agg(["mean", "std"])
low_earners_stats = df[df["salary K$"] <= 50]["age"].agg(["mean", "std"])

print(high_earners_stats)
print(low_earners_stats)

mean    43.959110
std     10.269633
Name: age, dtype: float64
mean    36.608060
std     13.464631
Name: age, dtype: float64


# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [76]:
educations = ["Bachelors", "Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"]
education_have_not = df[(df["education"].isin(educations) == False) & (df["salary"] == ">50K")]
education_have_not.shape[0]

3178

# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [77]:
statistics_age = df.groupby("education")["age"].describe()
statistics_age

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
10th,820.0,37.897561,16.225795,17.0,23.0,36.0,52.0,90.0
11th,1048.0,32.363550,15.089307,17.0,18.0,28.5,43.0,90.0
12th,377.0,32.013263,14.373710,17.0,19.0,28.0,41.0,79.0
1st-4th,151.0,44.622517,14.929051,19.0,33.0,44.0,56.0,81.0
5th-6th,288.0,41.649306,14.754622,17.0,28.0,41.0,53.0,82.0
7th-8th,557.0,47.631957,15.737479,17.0,34.0,49.0,60.0,90.0
9th,455.0,40.303297,15.335754,17.0,28.0,38.0,53.0,90.0
Assoc-acdm,1008.0,37.286706,10.509755,19.0,29.0,36.0,44.0,90.0
Assoc-voc,1307.0,38.246366,11.181253,19.0,30.0,37.0,45.0,84.0


# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [92]:
df["is_married"] = df["marital-status"].str.startswith("Married")
married_average_salary = df[(df["sex"] == "Male") & (df["is_married"] == True)]["salary K$"].mean()
not_married_average_salary = df[(df["sex"] == "Male") & (df["is_married"] == False)]["salary K$"].mean()

married_average_salary, not_married_average_salary

(np.float64(107.49455968688845), np.float64(46.59723865877712))

# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [95]:
max_hours = df["hours-per-week"].max()
count_people = df[df["hours-per-week"] == max_hours].shape[0]

max_hours, count_people

(np.int64(99), 78)

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [98]:
numeric_df = df.select_dtypes(include="number")
corr_matrix = numeric_df.corr()
corr_matrix

,age,hours-per-week,salary K$
age,1.000000,0.101599,0.208203
hours-per-week,0.101599,1.000000,0.196378
salary K$,0.208203,0.196378,1.000000
